## Load spread sheet

In [ ]:
%matplotlib inline
from typing import Iterable

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import NMF, PCA
from sklearn.model_selection import train_test_split

RANDOM_STATE = 1234
np.random.seed(RANDOM_STATE)

In [ ]:
# Load data from spreadsheet.
data_frame = pd.read_excel(
    '2019-08-27_PLASMA_DEFAULT_Results_Groningen.xlsx',
    sheet_name=2,
)

# Load the phenotypes from SPSS file.
phenotypes = pd.read_spss('phenotypes.sav')
# Remove patients for which there is no sequencing data.
phenotypes = phenotypes[~phenotypes['VAR00001'].isna()]
# And set Patient ID as index.
phenotypes['VAR00001'].name = 'Patient ID'
phenotypes.set_index(phenotypes['VAR00001'].astype(int), inplace=True)

To make a fair comparison we should split the data in a training and validation set by randomly selecting patients. 

In [ ]:
# Fraction of dataset we want to use for the validation set.
f_val = 0.3

patients = data_frame['Patient ID'].unique()
np.random.shuffle(patients)

# Determine the largest element no longer in the training set.
cut_off = int(np.ceil(len(patients) * (1 - f_val)))
train_patients = patients[:cut_off]
test_patients = patients[cut_off:]

# Put records in train/validation set according to "Patient ID".
train_rows = data_frame['Patient ID'].isin(train_patients)
train_data_frame = data_frame.loc[train_rows]
test_data_frame = data_frame[~train_rows]

## Aggregated statistics
Let us first look at the data as a whole (not just the training data), to explore the dataset.

In [ ]:
# Focus only on the genes
columns_to_keep = ['Patient ID', 'Gene']
data_frame = data_frame[columns_to_keep]

In [ ]:
# Count occurences of genes, regardless of how many per patient.
gene_set = data_frame['Gene'].unique()
gene_count  = data_frame.groupby('Gene') \
    .count()

print(gene_set)
# Plot occurences of genes, and make all genes with more than 3 instances red.
gene_colour = gene_count['Patient ID'].apply(lambda x: 'red' if x > 5 else 'blue')
ax = gene_count['Patient ID'].plot(kind='bar', color=gene_colour)
ax.set_ylabel('# occurences')

Clearly, the majority of the mutations are unique, whilst the presence of TP53 is quite ubiquitous. 

How are the mutations distributed? One per patient?

In [ ]:
# How many mutations per patient?
mutation_counts = data_frame.groupby('Patient ID').count()
ax = sns.distplot(mutation_counts, kde=False)
ax.set_xlabel('# Mutations')
ax.set_ylabel('Frequency')
ax.set_xlim([1, max(mutation_counts['Gene'])])

The figure above indicates that patients usually have one or two mutations. 

It can happen that there are multiple mutations in the same gene. 
How often does this occur?

In [ ]:
# How many patients have more than 1 mutation in the same gene?
# Group by (patient, gene):
num_gene_mutations = data_frame.groupby(columns_to_keep) \
    .size() \
    .to_frame('size')
# More than 1.
same_gene_mutations = num_gene_mutations[num_gene_mutations['size'] > 1]

# Fraction of total.
num_patients_multi_mutation = len(same_gene_mutations.groupby('Patient ID'))
num_patients = data_frame['Patient ID'].nunique()
f = num_patients_multi_mutation / num_patients

print('Number of patients with multiple mutations in same gene: {}/{} ({:.2f} %)'.format(
    num_patients_multi_mutation, 
    num_patients, 
    f * 100.0),
)

same_gene_mutations

But how many mutations, and what mutations?

In [ ]:
# Plot distribution of patients with at least two mutations.
ax = sns.distplot(
    same_gene_mutations, 
    kde=False,
    bins=[2, 3, 4, 5],
)
ax.set_xlabel('# mutations in single gene')
ax.set_ylabel('# patients')

# Multiple mutations in same gene occurs almost solely in TP53.
same_gene_mutations.groupby('Gene').describe()

## TODO
- correlations

## Text analysis

We can borrow some tricks that are also used in text analysis. For example, in the bag-of-words approach one collects all words (the vocabulary) and counts the occurences of each word per text document. I will do the same below: with _gene_ $\leftrightarrow$ _word_ and _text document_ $\leftrightarrow$ _patient_. That is, for each patient (row) count the number of mutations per gene (column). 

In [ ]:
def dummy_encode_mutations(
    data_frame: pd.DataFrame, gene_vocabulary: Iterable
) -> pd.DataFrame:
    """
    Dummy encode mutations for each patient.
    """
    # Create an empty data frame first.
    dummy_data_frame = pd.DataFrame(
        0,
        # Rows are patients
        index=data_frame["Patient ID"].unique(),
        # Columns are the number of mutations per gene.
        columns=gene_vocabulary,
    )
    dummy_data_frame.index.name = "Patient ID"

    # Fill all columns with the number of occurences of each gene per patient.
    for multi_index, patient_genes in data_frame.groupby(["Patient ID", "Gene"]):
        # Unpack patient_id and the gene from the `MultiIndex`.
        patient_id, gene = multi_index
        # Store number of mutations of this particular gene.
        dummy_data_frame.loc[patient_id][gene] += len(patient_genes)

    return dummy_data_frame

In [ ]:
# Vocabulary is the entire dataset, not only training set. Otherwise we run into problems during inference.
gene_vocabulary = data_frame['Gene'].unique()

dummy_data_frame = dummy_encode_mutations(train_data_frame, gene_vocabulary)

Combine with phenotype data

In [ ]:
phenotypes_to_keep = ['Clinical_Response', 'response_grouped', 'leeftijd']
df_with_phenotype = pd.merge(
    left=dummy_data_frame,
    right=phenotypes[phenotypes_to_keep],
    left_index=True,
    right_index=True,
)

In [ ]:
train_data_frame['Patient ID'].unique()
# train_data_frame.set_index('Patient ID').loc[1022]

In [ ]:
# Validate that the code above is correct.
assert dummy_data_frame.loc[1022]['TP53'] == 2
assert dummy_data_frame.loc[1172]['TP53'] == 4
assert dummy_data_frame.loc[1172]['STK11'] == 2

Non-negative matrix factorisation
$$X = WH$$

In [ ]:
nmf_decomp = NMF(n_components=2).fit(dummy_data_frame)
W = nmf_decomp.transform(dummy_data_frame)
# Add jittering to help visualisation.
W += np.random.normal(scale=0.025, size=W.shape)
sns.scatterplot(W[:,0], W[:,1], hue=df_with_phenotype['Clinical_Response'] == 'SD', x_jitter=True, y_jitter=True)

In [ ]:
pca_decomp = PCA(n_components=2).fit(dummy_data_frame)
L = pca_decomp.transform(dummy_data_frame)
# Add jittering to help visualisation.
L += np.random.normal(scale=0.075, size=L.shape)
sns.scatterplot(L[:,0], L[:,1], hue=df_with_phenotype['Clinical_Response'] == 'SD')